# Budget Analysis

---

## Environment Setup

In [1]:
# Initial Imports
import os
import plaid
import requests
import datetime
import json
import pandas as pd
from dotenv import load_dotenv

%matplotlib inline

In [75]:
# Helper functions

def pretty_print_response(response):
    """
    This function formats a JSON file to pretty printing it in the screen.
    """
    print(json.dumps(response, indent=4, sort_keys=True))



In [3]:
# Set environment variables from the .env file
load_dotenv()

True

In [4]:
# Extract API keys from environment variables
PLAID_CLIENT_ID = os.getenv("PLAID_CLIENT_ID")
PLAID_SBX_SECRET_KEY = os.getenv("PLAID_SBX_SECRET_KEY")
PLAID_PUBLIC_KEY = os.getenv("PLAID_PUBLIC_KEY")

---

## Plaid Access Token

In this section, you will use the `plaid-python` API to generate the correct authentication tokens to access data in the free developer Sandbox. This mimics how you might connect to your own account or a customer account, but due to privacy issues, this homework will only require connecting to and analyzing the fake data from the developer sandbox that Plaid provides. 

Complete the following steps to generate an access token:

1. Create a client to connect to plaid

2. Use the client to generate a public token and request the following items: ['transactions', 'income', 'assets']

3. Exchange the public token for an access token

4. Test the access token by requesting and printing the available test accounts

### 1. Create a client to connect to plaid

In [5]:
# Create the client object
client = plaid.Client(client_id=PLAID_CLIENT_ID, secret=PLAID_SBX_SECRET_KEY, public_key=PLAID_PUBLIC_KEY, environment='sandbox')


In [6]:
# Set the institution id
INSTITUTION_ID = "ins_109508"

### 2. Generate a public token

In [7]:
# Create the public token
tkn_response = client.Sandbox.public_token.create(INSTITUTION_ID,['transactions', 'income', 'assets'])

# Print the public token
print(tkn_response)

# Are these public/access token guidelines outlined in the Plaid API documentation on their website?

{'public_token': 'public-sandbox-5ae4d009-973a-4316-952b-cbb836408f9e', 'request_id': 'WHa2r9W3qN9IwGK'}


### 3. Exchange the public token for an access token

In [8]:
# Retrieve the access token
exchange_response = client.Item.public_token.exchange(tkn_response['public_token'])


# Print the access token
print(exchange_response['access_token'])

access-sandbox-cd5ce729-4884-428d-a303-7cd45c3c437a


In [9]:
# Store the access token in a Python variable
access_token = exchange_response['access_token']

### 4. Fetch Accounts

In [76]:
# Retrieve accounts

start_date = '{:%Y-%m-%d}'.format(datetime.datetime.now() + datetime.timedelta(-90))
# Set time delta to '-90' instead of '-365' to indicate the last 90 days.

end_date = '{:%Y-%m-%d}'.format(datetime.datetime.now()) 

transaction_response = client.Transactions.get(access_token, start_date, end_date)

# Print accounts
#print(json.dumps(transaction_response['transactions'][1], indent=4, sort_keys=True))
#print(transaction_response.values())
key_list = [key for key in transaction_response]
print(key_list)

print(pretty_print_response(transaction_response))
#type(transaction_response)

['accounts', 'item', 'request_id', 'total_transactions', 'transactions']
{
    "accounts": [
        {
            "account_id": "MM4J3dGqAQt8B581jXeluEXxGaazMxU91w4qN",
            "balances": {
                "available": 100,
                "current": 110,
                "iso_currency_code": "USD",
                "limit": null,
                "unofficial_currency_code": null
            },
            "mask": "0000",
            "name": "Plaid Checking",
            "official_name": "Plaid Gold Standard 0% Interest Checking",
            "subtype": "checking",
            "type": "depository"
        },
        {
            "account_id": "1pVQk73ZGgIx8qx1PNbZUwVjZrrxzji5rLqBD",
            "balances": {
                "available": 200,
                "current": 210,
                "iso_currency_code": "USD",
                "limit": null,
                "unofficial_currency_code": null
            },
            "mask": "1111",
            "name": "Plaid Saving",
         

---

# Account Transactions with Plaid

In this section, you will use the Plaid Python SDK to connect to the Developer Sandbox account and grab a list of transactions. You will need to complete the following steps:


1. Use the access token to fetch the transactions for the last 90 days

2. Print the categories for each transaction type

3. Create a new DataFrame using the following fields from the JSON transaction data: `date, name, amount, category`. (For categories with more than one label, just use the first category label in the list)

4. Convert the data types to the appropriate types (i.e. datetimeindex for the date and float for the amount)

### 1. Fetch the Transactions for the last 90 days

In [77]:
# Set the start and end date 90 days from today

# Retrieve the transactions for the last 90 days
transactions = json.dumps(transaction_response['transactions'], indent=4, sort_keys=True)

# Print the transactions
print(type(transactions))
print(transactions)

<class 'str'>
[
    {
        "account_id": "pVDvxkpwzQS5dz5l9mwpsnJZa556DZCLzyJa8",
        "account_owner": null,
        "amount": 500,
        "authorized_date": null,
        "category": [
            "Food and Drink",
            "Restaurants"
        ],
        "category_id": "13005000",
        "date": "2020-07-24",
        "iso_currency_code": "USD",
        "location": {
            "address": null,
            "city": null,
            "country": null,
            "lat": null,
            "lon": null,
            "postal_code": null,
            "region": null,
            "store_number": null
        },
        "merchant_name": "Tectra Inc",
        "name": "Tectra Inc",
        "payment_channel": "in store",
        "payment_meta": {
            "by_order_of": null,
            "payee": null,
            "payer": null,
            "payment_method": null,
            "payment_processor": null,
            "ppd_id": null,
            "reason": null,
            "reference_nu

### 2. Print the categories for each transaction

In [95]:
# Create a for-loop to print the categories for each transaction
#categories = [category for category in transactions]
transactions_json = json.loads(transactions)
print(type(transactions_json))
categories = [category for category in transactions_json[0]]
categories


<class 'list'>


['account_id',
 'account_owner',
 'amount',
 'authorized_date',
 'category',
 'category_id',
 'date',
 'iso_currency_code',
 'location',
 'merchant_name',
 'name',
 'payment_channel',
 'payment_meta',
 'pending',
 'pending_transaction_id',
 'transaction_code',
 'transaction_id',
 'transaction_type',
 'unofficial_currency_code']

### 3. Create a new DataFrame using the following fields from the JSON transaction data: `date`, `name`, `amount`, `category`. 

(For categories with more than one label, just use the first category label in the list)

In [ ]:
# Define an empty DataFrame and set the columns as date, name, amount, and category
df = pd.Dataframe()

# Retrieve all the transactions

# Populate the transactions DataFrame with the transactions data

# Display sample data from the DataFrame    


### 4. Convert the data types to the appropriate types 

(i.e. datetimeindex for the date and float for the amount)

In [38]:
# Display data types
dict = {'asdf': 122, 'qwer': 123}
print(dict.values())
values_list = [dict[key] for key in dict]
values_list

dict_values([122, 123])


[122, 123]

In [ ]:
# Make any required data type transformation


In [ ]:
# Set the date column as index

# Display sample data


---

# Income Analysis with Plaid

In this section, you will use the Plaid Sandbox to complete the following:

1. Determine the previous year's gross income and print the results

2. Determine the current monthly income and print the results

3. Determine the projected yearly income and print the results

In [ ]:
# Fetch de income data

# Print the income data


In [ ]:
# Determine the previous year's gross income and print the results


In [ ]:
# Determine the current monthly income and print the results


In [ ]:
# Determine the projected yearly income and print the results


---

# Budget Analysis
In this section, you will use the transactions DataFrame to analyze the customer's budget

1. Calculate the total spending per category and print the results (Hint: groupby or count transactions per category)

2. Generate a bar chart with the number of transactions for each category

3. Calulate the expenses per month

4. Plot the total expenses per month

### Calculate the expenses per category

In [ ]:
# Compute the total expenses per category

# Display the expenses per catefory


In [ ]:
# Create a spending categories pie chart.


### Calculate the expenses per month

In [ ]:
# Create a DataFrame with the total expenses

# Display sample data


In [ ]:
# Create a spending per month bar chart
